# 0) Packages and functions

In [1]:
import pandas as pd
import numpy as np
import calendar
import time
import glob
import os
import scipy

#Disable the 'SettingWithCopyWarning' when adding new columns to an existing dataframe
pd.options.mode.chained_assignment = None  # default='warn'

#Data cleaning
from functools import reduce
import unicodedata
import re

#Ignore warnings about the DataFrameGroupBy() method
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

#For the graphics
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
import matplotlib.patches as mpatches

#Feature selection
from sklearn.inspection import permutation_importance

#Modeling
from sklearn.model_selection import cross_validate
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ParameterGrid   #to implement manual GridSearch
#-> RF
from sklearn.ensemble import RandomForestRegressor
#-> GBR
from sklearn.ensemble import GradientBoostingRegressor
#-> CatBoost
#!pip install catboost
#import catboost
#-> LightGBM
#!pip install LightGBM
#import lightgbm
#-> XGBoost
#!pip install XGBoost
import xgboost

#Metrics
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.metrics import make_scorer   #needed to use MASE on the cross validation stages

#Dynamic time warping: grouping
from dtaidistance import dtw
from dtaidistance import dtw_ndim

In [2]:
def describe(df, data_description, anomaly_any, stats=['skew', 'median']):
  '''
  Custom describe function. Adds more statistics, which were used to complement the
  mean and best characterize the resutls, to the usual pd.describe()

  df: Dataframe containing the results to be analysed
  data_description: string used during simulations. Used to filter each time series
  -> selection criteria (such as GDP) from the complete results dataframe.
  anomaly_any: binary. 0 is used to filter results, while 1 shows data for all tested cities.
  stats: skewness and median as default, but could accept more entries (see statistics implemented in the Pandas package).
  '''

  #Filters by data description
  df = df.loc[df['data_description'] == data_description]
  if (anomaly_any == 0):    #selects data for cities without anomalies
    df = df.loc[df['anomaly_any'] == anomaly_any]
  #Define custom percentiles. My argument here is that we still get results better than the naïve model on up to 85% of all data
  d = df.describe(percentiles=[.25, .5, .85])
  d.drop(columns=['cod_city', 'exec_time'], inplace=True)   #show only MASE and MAE
  aux = df.reindex(d.columns, axis = 1).agg(stats)
  d = pd.concat([d, aux])

  return d

Accuracy evaluation

In [3]:
def mean_absolute_scaled_error(y_true, y_pred, y_train):
    """
    Computes the MEAN-ABSOLUTE SCALED ERROR forecast error for univariate time series prediction.

    See "Another look at measures of forecast accuracy", Rob J Hyndman

    parameters:
        y_train: the series used to train the model, 1d numpy array
        y_true: the test series to predict, 1d numpy array or float
        y_pred: the prediction of y_true, 1d numpy array (same size as y_true) or float
        absolute: "squares" to use sum of squares and root the result, "absolute" to use absolute values.

    """
    n = y_train.shape[0]
    d = np.abs( np.diff( y_train) ).sum()/(n-1)

    errors = mean_absolute_error(y_true, y_pred)
    return errors.mean()/d

In [4]:
def best_model_GridSearchCV(param_grid, X_train, y_train, splits, base_estimator):
  start_run = time.time()
  ts_cv = TimeSeriesSplit(n_splits=splits)
  mase_scorer = make_scorer(mean_absolute_scaled_error, greater_is_better=False, y_train=y_train)
  scorer = {'mae': 'neg_mean_absolute_error', 'mase': mase_scorer}

  sh = GridSearchCV(base_estimator, param_grid, scoring=scorer, refit='mase', cv=splits, n_jobs=-1).fit(X_train, y_train)
  best_model = sh.best_estimator_
  best_params = sh.best_params_

  cv_results = cross_validate(
    best_model,
    X_train,
    y_train,
    cv=ts_cv,
    scoring=scorer,
    return_train_score=True
  )
  #Train
  train_mae = -cv_results["train_mae"]
  train_mase = -cv_results["train_mase"]

  #Val
  val_mae = -cv_results["test_mae"]
  val_mase = -cv_results["test_mase"]

  end_run = time.time()
  runtime = round( (end_run - start_run), 2)

  return best_model, best_params, runtime, train_mae.mean(), train_mase.mean(), val_mae.mean(), val_mase.mean()

Anomaly detection

In [5]:
def detect_anomalies_test(data, name_pred, test_size=0.2):
  df_city = data.loc[data['cod_city'] == name_pred]
  y = df_city[df_city.columns[7]]
  df_city = df_city.drop(columns=list(df_city.columns[:7]))
  X = df_city[df_city.columns[1:]]    #Not really necessary to the evaluation, but the split needs it

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42, shuffle=False)

  # calculate mean
  mean = np.mean(y_train.values)
  # calculate standard deviation
  sd = np.std(y_train.values)
  # determine a threshold
  threshold = 4   #4 sigma

  #Count the number of zeros in the train set
  zeros = (y_train.values == 0).astype(int).sum() / len(y_train)

  #Initialize response variables
  z_score = []
  anomaly_class = 0

  # detect outlier
  for i in y_test.values:
    z = (i-mean)/sd # calculate z-score
    if abs(z) > threshold:  # identify outliers
      anomaly_class = 1   #it's enough to know that there's at least one outlier in the test set
      z_score.append(z)   #add all z scores to a list

  if (len(z_score) == 0):
    z_score = 0
    return anomaly_class, z_score, zeros
  else:
    return anomaly_class, np.mean(z_score), round(zeros, 3)

Train-test split

In [48]:
def get_train_test_extended(disease, data, distances_data, cod_city, agg_type, number_of_cities, scaled=False):
  '''
  distances_data: Pandas Dataframe. Assumes the columns 'city_1', city_2' and 'distance', in this order.
  agg_type: 'topk' or 'topk_avg'
  '''
  #Filter the data according to the chosen division - legacy (left here for debugging purposes)
  topk = list(distances_data.loc[distances_data['city_1'] == cod_city]['city_2'].values)
  topk = topk[:number_of_cities]

  data_filtered = data.loc[data['cod_city'].isin(topk)]

  #Get the cities contained in the division
  cities = list(data_filtered.cod_city.unique())
  city_counter = 1

  #Select the city being studied
  if (disease == 'covid'):
    index_cases = 7
  if ((disease == 'zika') or (disease == 'influenza')):
    index_cases = 5
  if (disease == 'dengue'):
    index_cases = 9

  X_extra = pd.DataFrame()
  for city in cities:
    swp = data_filtered.loc[data_filtered['cod_city'] == city]
    swp = swp.reset_index()
    swp.drop(columns='index', inplace=True)

    #rename the columns to make each data column from the cities unique.
    name_columns = list(swp.columns)
    renamed_columns = ['nb' + str(city_counter) + '_' + category for category in name_columns]
    swp.columns = renamed_columns
    swp = swp[swp.columns[index_cases]]
    X_extra = pd.concat([X_extra, swp], axis=1)
    city_counter = city_counter + 1

  df_city = data.loc[data['cod_city'] == cod_city]
  y = df_city[df_city.columns[index_cases]]   #this split respects the structure of the dataset containing diseases cases after using the data prep notebook.
  df_city = df_city.drop(columns=list(df_city.columns[:index_cases]))
  X = df_city[df_city.columns[1:]]    #lags for the city being studied, features

  if (agg_type == 'topk'):
    X = X.reset_index()
    X.drop(columns='index', inplace=True)
    X = pd.concat([X, X_extra], axis=1)

  if (agg_type == 'topk_avg'):
    X_swp = pd.DataFrame()
    for n in range(1, len(cities)+1):
      X_columns_city = list(X_extra.filter(regex=str('nb' + str(n) + '_')))
      swp2 = X_extra[X_columns_city].mean(axis=1)
      swp2 = swp2.to_frame(name=str('nb' + str(n) + '_lag_avg'))
      X_swp = pd.concat([X_swp, swp2], axis=1)

    X = X.reset_index()
    X.drop(columns='index', inplace=True)
    X = pd.concat([X, X_swp], axis=1)

  #After assembling the dataframe, apply the train/test split.
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                      random_state=42, shuffle=False)   #1/15: the test size will be made of 1 year of observations
  if(scaled == True):
    scaler_series = preprocessing.MinMaxScaler()
    X_train = scaler_series.fit_transform(X_train)
    X_test = scaler_series.transform(X_test)
    return X_train, X_test, y_train, y_test, scaler_series
  else:
    return X_train, X_test, y_train, y_test

In [7]:
def get_train_test_single(disease, data, cod_city, scaled=False):
  '''
  This code selects the data of the city (cod_city) being studied and converts it
  to a darTS input.
  '''

  #Select the city being studied
  df_city = data.loc[data['cod_city'] == cod_city]
  df_city.reset_index(inplace=True, drop=True)

  #Select the city being studied
  if (disease == 'zika'):
    index_cases = 9
  if ((disease == 'zika') or (disease == 'influenza')):
    index_cases = 7
  if (disease == 'dengue'):
    index_cases = 11
  X = df_city[df_city.columns[index_cases+1:]]  #this split respects the structure of the dataset containing diseases cases after using the data prep notebook.
  y = df_city[df_city.columns[index_cases]]

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                      random_state=42, shuffle=False)   #1/15: the test size will be made of 1 year (2022) of observations
  if(scaled):
    scaler_series = preprocessing.MinMaxScaler()
    X_train = scaler_series.fit_transform(X_train)
    X_test = scaler_series.transform(X_test)
    return X_train, X_test, y_train, y_test, scaler_series
  else:
    return X_train, X_test, y_train, y_test

Function to execute direct, one-to-one, predictions

In [8]:
def run_model(disease, model, data, distances_data, data_description, pred_type, name_pred,
              number_of_cities, scaled=True):
  '''
  model: string to determine which model to use for regression. Options: RF, GBR, CatBoost, LightGBM, XGBoost
  X and y: features and target, respectively, to be used, which should be generated with closest_cities().
  data_description: can be a loose string. I use it latter to aggregate results on graphics.
  name_pred: string describing the city or region being analysed
  scaled: boolean. Wheter to use scaled train/test sets or not.
  '''
  #Model selection
  if (model == 'RF'):
    grid = {'n_estimators': [25,50,100,150,200], 'max_depth': [2,4,None]}
    estimator = RandomForestRegressor(random_state=42)

  if (model == 'GBR'):
    grid = {'n_estimators': [25,50,100,150,200], 'max_depth': [2,4,None]}
    estimator = GradientBoostingRegressor(random_state=42)

  if (model == 'CatBoost'):
    grid = {'iterations': [25,50,100,150,200], 'depth': [2,4,None]}
    estimator = catboost.CatBoostRegressor(random_state=42, silent=True)   #silent: This model will have a lot to say otherwise.

  if (model == 'LightGBM'):
    grid = {'n_estimators': [50,100,150], 'max_depth': [2,4,None]}
    estimator = lightgbm.LGBMRegressor(random_state=42)

  if (model == 'XGBoost'):
    grid = {'n_estimators': [25,50,100,150,200], 'learning_rate': [0.001, 0.005, 0.01],
            'max_depth': [2,4,None]}
    estimator = xgboost.XGBRegressor(random_state=42)

  #Splitting the datasets
  if (pred_type == 'single'):
    X_train, X_test, y_train, y_test, _ = get_train_test_single(disease, data, name_pred, scaled=scaled)
  if ((pred_type == 'topk') or (pred_type == 'topk_avg')):
    X_train, X_test, y_train, y_test, _ = get_train_test_extended(disease, data, distances_data, name_pred, pred_type, number_of_cities, scaled=scaled)

  #[Train and val using CV] Model predictions on the best parameter combination found by exaustive GridSearch + CV
  model_CV, params, runtime, train_mae, train_mase, val_mae, val_mase = best_model_GridSearchCV(
      param_grid=grid, X_train=X_train, y_train=y_train, splits=4, base_estimator=estimator)

  par_names = [list(params.keys())]
  par_values = [list(params.values())]

  #[Test set, hold-out]
  model_CV.fit(X_train, y_train)
  y_pred_test = model_CV.predict(X_test)
  test_mae = mean_absolute_error(y_test, y_pred_test)
  test_mase = mean_absolute_scaled_error(y_test, y_pred_test, y_train)

  #Detect anomalies in the test set
  anomaly_class, mean_z_score, _ = detect_anomalies_test(data, name_pred, test_size=0.2)

  res = {'model': model, 'pred_for': pred_type, 'cod_city': name_pred,
         'data_description': data_description, 'par_names': par_names, 'par_values': par_values,
         'scaled?': scaled, 'exec_time': runtime, 'mae_train': train_mae,
         'mase_train': train_mase, 'mae_val': val_mae, 'mase_val': val_mase,
         'mae_test': test_mae, 'mase_test': test_mase, 'anomaly_any':anomaly_class,
         'z_score': mean_z_score}

  df_res = pd.DataFrame(res, index=[0])
  print(model, 'runtime:', runtime, 's.')

  return df_res

In [9]:
def display_model(model, data, distances_data, data_description, pred_type, name_pred,
              number_of_cities, scaled=True):
  '''
  Almost the same function as the previous one. Instead of displaying accuracies, here
  the results are the predicted time series. Useful for generating visualizations.
  '''
  #Model selection
  if (model == 'RF'):
    grid = {'n_estimators': [25,50,100,150,200], 'max_depth': [2,4,None]}
    estimator = RandomForestRegressor(random_state=42)

  if (model == 'GBR'):
    grid = {'n_estimators': [25,50,100,150,200], 'max_depth': [2,4,None]}
    estimator = GradientBoostingRegressor(random_state=42)

  if (model == 'CatBoost'):
    grid = {'iterations': [25,50,100,150,200], 'depth': [2,4,None]}
    estimator = catboost.CatBoostRegressor(random_state=42, silent=True)   #silent: This model will have a lot to say otherwise.

  if (model == 'LightGBM'):
    grid = {'n_estimators': [50,100,150], 'max_depth': [2,4,None]}
    estimator = lightgbm.LGBMRegressor(random_state=42)

  if (model == 'XGBoost'):
    grid = {'n_estimators': [25,50,100,150,200], 'learning_rate': [0.001, 0.005, 0.01],
            'max_depth': [2,4,None]}
    estimator = xgboost.XGBRegressor(random_state=42)

  #Splitting the datasets
  if (pred_type == 'single'):
    X_train, X_test, y_train, y_test, _ = get_train_test_single(data, name_pred, scaled=scaled)
  if ((pred_type == 'topk') or (pred_type == 'topk_avg')):
    X_train, X_test, y_train, y_test, _ = get_train_test_extended(data, distances_data, name_pred, pred_type, number_of_cities, scaled=scaled)
  if (pred_type == 'all'):
    X_train, X_test, y_train, y_test, _ = get_train_test_all(data, scaled=scaled)

  #[Train and val using CV] Model predictions on the best parameter combination found by exaustive GridSearch + CV
  model_CV, params, runtime, train_mae, train_mase, val_mae, val_mase = best_model_GridSearchCV(
      param_grid=grid, X_train=X_train, y_train=y_train, splits=4, base_estimator=estimator)

  par_names = [list(params.keys())]
  par_values = [list(params.values())]

  #[Test set, hold-out]
  model_CV.fit(X_train, y_train)
  y_pred_train = model_CV.predict(X_train)
  y_pred_test = model_CV.predict(X_test)

  return y_train, y_test, y_pred_train, y_pred_test

In [10]:
def get_city_names(df, list_codes):
  '''
  Quick function to extract names from the unique codes of the disease's datasets.
  '''
  list_strings = []

  for city in list_codes:
    filter = df.loc[df['cod_city'] == city]
    state = filter['state'].values[0]
    name_city = filter['city'].values[0]

    list_strings.append(name_city + ', ' + state)

  return list_strings

In [11]:
def generate_figures(disease, model, data, distances_data, number_of_cities,
                     list_cities, list_cities_names, save_path):
  '''
  Generate figures in batches according to the city list provided in list_cities.
  '''

  counter = 0
  for city in list_cities:
    #Run forecasts
    y_train, y_test, y_pred_train, y_pred_test = display_model(model=model, data=data, distances_data=distances_data,
                      data_description='None', pred_type='topk', name_pred=city,
                      number_of_cities=number_of_cities, scaled=True)

    #Plots and save the resulting figure comparing the dataset and forecasts
    plt.plot(list(range(len(y_train), len(y_train) + len(y_test), 1)), y_test.values, label='Actual', color='g')
    plt.plot(list(range(len(y_train), len(y_train) + len(y_test), 1)), y_pred_test, label='Prediction', color='orange')
    plt.plot(y_train.values, color='g')
    plt.plot(y_pred_train, color='orange')
    plt.legend(title=list_cities_names[counter], loc='upper left')
    plt.axvline(x=len(y_train), color= 'k', ls='--')
    plt.savefig(save_path + disease + '_' + model + '_' + str(city) + '.eps', format='eps')
    plt.show()

    counter = counter +1

# 0) Loading multiple dataframes

In [64]:
#Table with distances between cities.
disease = 'dengue'
data_path = 'E:/Doutorado/Trabalhos/Projeto-dengue/data/' + disease + '/'

df_geo_distances = pd.read_csv(data_path + disease + "_geo_distances_cities.csv")

#Table with distances between time series calculated with Dynamic Time Warping (DTW), on the data prep notebook
df_cases_dtw =  pd.read_csv(data_path + disease +'_distances_cases_dtw.csv')

#Another distance table, but using the PIB/GDP values to attribute similarity between the cities
df_pib_dtw = pd.read_csv(data_path + disease + '_pib_dtw.csv')

#Load the datasets with the first five lags
df = pd.read_csv(data_path + disease + '_first_5_lags.csv')
df.drop(columns='Unnamed: 0', inplace=True)

# 1) Direct predictions on cities

In [ ]:
#Parameter to use for the simulation
data_to_use = df
model_list = ['XGBoost', 'RF']
path = 'C:/Users/your_path/'

In [ ]:
#Run the simulation
df_res_cities = pd.DataFrame()

state_count = 0
num_states = len(data_to_use.state.unique())

for state in list(data_to_use.state.unique()):
  print('---', state , '---')
  print('State', state_count+1, 'of', num_states)
  city_count = 0
  single_state_data = data_to_use.loc[data_to_use['state'] == state]
  num_cities = len(single_state_data['cod_city'].unique())
  cities_list = list(single_state_data['cod_city'].unique())    #We'll be using city codes instead of their names

  for city in cities_list:
    print('---', city, '---')
    print('City', city_count+1, 'of', num_cities)
    for num_model in range(len(model_list)):
      swp = run_model(model=model_list[num_model], data=data_to_use, distances_data=None,
                      data_description='Single city', pred_type='single', name_pred=city,
                      number_of_cities=None, scaled=True)
      #Saves at each step as these calculations can take a while.
      df_res_cities = pd.concat([df_res_cities, swp])

    city_count += 1
  state_count +=1
df_res_cities.to_csv(path + disease + '_single_city_seq_lags.csv')

# 2) Selecting the top k cities with optimal distance

## Top 1

In [ ]:
#Parameters to use for all simulation
data_to_use = df
model_list = ['XGBoost', 'RF']
path = 'C:/Users/your_path/'

Geographical distances

In [ ]:
data_distances = df_geo_distances         #Table with geographical distances for each city

In [ ]:
#Run the simulation
df_res_cities = pd.DataFrame()

state_count = 0
num_states = len(data_to_use.state.unique())

for state in list(data_to_use.state.unique()):
  print('---', state , '---')
  print('State', state_count+1, 'of', num_states)
  city_count = 0
  single_state_data = data_to_use.loc[data_to_use['state'] == state]
  num_cities = len(single_state_data['cod_city'].unique())
  cities_list = list(single_state_data['cod_city'].unique())    #We'll be using city codes instead of their names

  for city in cities_list:
    print('---', city, '---')
    print('City', city_count+1, 'of', num_cities)
    for num_model in range(len(model_list)):
      swp = run_model(model=model_list[num_model], data=data_to_use, distances_data=data_distances,
                                      data_description='Geo, top 1', pred_type='topk', name_pred=city, number_of_cities=1, scaled=True)
      #Saves at each step as these calculations can take a while.
      df_res_cities = pd.concat([df_res_cities, swp])

    city_count += 1
  state_count +=1
df_res_cities.to_csv(path + disease + '_top1_Geo_XGBoost.csv')

DTW distances on dengue cases

In [ ]:
data_distances = df_cases_dtw         #Table with geographical distances for each city

In [ ]:
#Run the simulation
df_res_cities = pd.DataFrame()

state_count = 0
num_states = len(data_to_use.state.unique())

for state in list(data_to_use.state.unique()):
  print('---', state , '---')
  print('State', state_count+1, 'of', num_states)
  city_count = 0
  single_state_data = data_to_use.loc[data_to_use['state'] == state]
  num_cities = len(single_state_data['cod_city'].unique())
  cities_list = list(single_state_data['cod_city'].unique())    #We'll be using city codes instead of their names

  for city in cities_list:
    print('---', city, '---')
    print('City', city_count+1, 'of', num_cities)
    for num_model in range(len(model_list)):
      swp = run_model(model=model_list[num_model], data=data_to_use, distances_data=data_distances,
                                      data_description='TS, top 1', pred_type='topk', name_pred=city, number_of_cities=1, scaled=True)
      #Saves at each step as these calculations can take a while.
      df_res_cities = pd.concat([df_res_cities, swp])

    city_count += 1
  state_count +=1
df_res_cities.to_csv(path + disease + '_top1_TS_XGBoost.csv')

GDP distances

In [ ]:
data_distances = df_pib_dtw         #Table with geographical distances for each city

In [ ]:
#Run the simulation
df_res_cities = pd.DataFrame()

state_count = 0
num_states = len(data_to_use.state.unique())

for state in list(data_to_use.state.unique()):
  print('---', state , '---')
  print('State', state_count+1, 'of', num_states)
  city_count = 0
  single_state_data = data_to_use.loc[data_to_use['state'] == state]
  num_cities = len(single_state_data['cod_city'].unique())
  cities_list = list(single_state_data['cod_city'].unique())    #We'll be using city codes instead of their names

  for city in cities_list:
    print('---', city, '---')
    print('City', city_count+1, 'of', num_cities)
    for num_model in range(len(model_list)):
      swp = run_model(model=model_list[num_model], data=data_to_use, distances_data=data_distances,
                                      data_description='GDP, top 1', pred_type='topk', name_pred=city, number_of_cities=1, scaled=True)
      #Saves at each step as these calculations can take a while.
      df_res_cities = pd.concat([df_res_cities, swp])

    city_count += 1
  state_count +=1
df_res_cities.to_csv(path + disease + '_top1_GDP_XGBoost.csv')

## Top 2

In [ ]:
#Parameters to use for all simulation
data_to_use = df
model_list = ['XGBoost', 'RF']
path = 'C:/Users/your_path/'

Geographical distances

In [ ]:
data_distances = df_geo_distances         #Table with geographical distances for each city

In [ ]:
#Run the simulation
df_res_cities = pd.DataFrame()

state_count = 0
num_states = len(data_to_use.state.unique())

for state in list(data_to_use.state.unique()):
  print('---', state , '---')
  print('State', state_count+1, 'of', num_states)
  city_count = 0
  single_state_data = data_to_use.loc[data_to_use['state'] == state]
  num_cities = len(single_state_data['cod_city'].unique())
  cities_list = list(single_state_data['cod_city'].unique())    #We'll be using city codes instead of their names

  for city in cities_list:
    print('---', city, '---')
    print('City', city_count+1, 'of', num_cities)
    for num_model in range(len(model_list)):
      swp = run_model(model=model_list[num_model], data=data_to_use, distances_data=data_distances,
                                      data_description='Geo, top 2', pred_type='topk', name_pred=city, number_of_cities=2, scaled=True)
      #Saves at each step as these calculations can take a while.
      df_res_cities = pd.concat([df_res_cities, swp])

    city_count += 1
  state_count +=1
df_res_cities.to_csv(path + disease + '_top2_Geo_XGBoost.csv')

DTW distances on dengue cases

In [ ]:
data_distances = df_cases_dtw         #Table with geographical distances for each city

In [ ]:
#Run the simulation
df_res_cities = pd.DataFrame()

state_count = 0
num_states = len(data_to_use.state.unique())

for state in list(data_to_use.state.unique()):
  print('---', state , '---')
  print('State', state_count+1, 'of', num_states)
  city_count = 0
  single_state_data = data_to_use.loc[data_to_use['state'] == state]
  num_cities = len(single_state_data['cod_city'].unique())
  cities_list = list(single_state_data['cod_city'].unique())    #We'll be using city codes instead of their names

  for city in cities_list:
    print('---', city, '---')
    print('City', city_count+1, 'of', num_cities)
    for num_model in range(len(model_list)):
      swp = run_model(model=model_list[num_model], data=data_to_use, distances_data=data_distances,
                                      data_description='TS, top 2', pred_type='topk', name_pred=city, number_of_cities=2, scaled=True)
      #Saves at each step as these calculations can take a while.
      df_res_cities = pd.concat([df_res_cities, swp])

    city_count += 1
  state_count +=1
df_res_cities.to_csv(path + disease + '_top2_TS_XGBoost.csv')

GDP distances

In [ ]:
data_distances = df_pib_dtw         #Table with geographical distances for each city

In [ ]:
#Run the simulation
df_res_cities = pd.DataFrame()

state_count = 0
num_states = len(data_to_use.state.unique())

for state in list(data_to_use.state.unique()):
  print('---', state , '---')
  print('State', state_count+1, 'of', num_states)
  city_count = 0
  single_state_data = data_to_use.loc[data_to_use['state'] == state]
  num_cities = len(single_state_data['cod_city'].unique())
  cities_list = list(single_state_data['cod_city'].unique())    #We'll be using city codes instead of their names

  for city in cities_list:
    print('---', city, '---')
    print('City', city_count+1, 'of', num_cities)
    for num_model in range(len(model_list)):
      swp = run_model(model=model_list[num_model], data=data_to_use, distances_data=data_distances,
                                      data_description='GDP, top 2', pred_type='topk', name_pred=city, number_of_cities=2, scaled=True)
      #Saves at each step as these calculations can take a while.
      df_res_cities = pd.concat([df_res_cities, swp])

    city_count += 1
  state_count +=1
df_res_cities.to_csv(path + disease + '_top2_GDP_XGBoost.csv')

## Top 3

In [ ]:
#Parameters to use for all simulation
data_to_use = df
model_list = ['XGBoost', 'RF']
path = 'C:/Users/your_path/'

Geographical distances

In [ ]:
data_distances = df_geo_distances         #Table with geographical distances for each city

In [ ]:
#Run the simulation
df_res_cities = pd.DataFrame()

state_count = 0
num_states = len(data_to_use.state.unique())

for state in list(data_to_use.state.unique()):
  print('---', state , '---')
  print('State', state_count+1, 'of', num_states)
  city_count = 0
  single_state_data = data_to_use.loc[data_to_use['state'] == state]
  num_cities = len(single_state_data['cod_city'].unique())
  cities_list = list(single_state_data['cod_city'].unique())    #We'll be using city codes instead of their names

  for city in cities_list:
    print('---', city, '---')
    print('City', city_count+1, 'of', num_cities)
    for num_model in range(len(model_list)):
      swp = run_model(model=model_list[num_model], data=data_to_use, distances_data=data_distances,
                                      data_description='Geo, top 3', pred_type='topk', name_pred=city, number_of_cities=3, scaled=True)
      #Saves at each step as these calculations can take a while.
      df_res_cities = pd.concat([df_res_cities, swp])

    city_count += 1
  state_count +=1
df_res_cities.to_csv(path + disease + '_top3_Geo_XGBoost.csv')

DTW distances on dengue cases

In [ ]:
data_distances = df_cases_dtw         #Table with geographical distances for each city

In [ ]:
#Run the simulation
df_res_cities = pd.DataFrame()

state_count = 0
num_states = len(data_to_use.state.unique())

for state in list(data_to_use.state.unique()):
  print('---', state , '---')
  print('State', state_count+1, 'of', num_states)
  city_count = 0
  single_state_data = data_to_use.loc[data_to_use['state'] == state]
  num_cities = len(single_state_data['cod_city'].unique())
  cities_list = list(single_state_data['cod_city'].unique())    #We'll be using city codes instead of their names

  for city in cities_list:
    print('---', city, '---')
    print('City', city_count+1, 'of', num_cities)
    for num_model in range(len(model_list)):
      swp = run_model(model=model_list[num_model], data=data_to_use, distances_data=data_distances,
                                      data_description='TS, top 3', pred_type='topk', name_pred=city, number_of_cities=3, scaled=True)
      #Saves at each step as these calculations can take a while.
      df_res_cities = pd.concat([df_res_cities, swp])

    city_count += 1
  state_count +=1
df_res_cities.to_csv(path + disease + '_top3_TS_XGBoost.csv')

GDP distances

In [ ]:
data_distances = df_pib_dtw         #Table with geographical distances for each city

In [ ]:
#Run the simulation
df_res_cities = pd.DataFrame()

state_count = 0+6
num_states = len(data_to_use.state.unique())

for state in list(data_to_use.state.unique()):
  print('---', state , '---')
  print('State', state_count+1, 'of', num_states)
  city_count = 0
  single_state_data = data_to_use.loc[data_to_use['state'] == state]
  num_cities = len(single_state_data['cod_city'].unique())
  cities_list = list(single_state_data['cod_city'].unique())    #We'll be using city codes instead of their names

  for city in cities_list:
    print('---', city, '---')
    print('City', city_count+1, 'of', num_cities)
    for num_model in range(len(model_list)):
      swp = run_model(model=model_list[num_model], data=data_to_use, distances_data=data_distances,
                                      data_description='GDP, top 3', pred_type='topk', name_pred=city, number_of_cities=3, scaled=True)
      #Saves at each step as these calculations can take a while.
      df_res_cities = pd.concat([df_res_cities, swp])

    city_count += 1
  state_count +=1
df_res_cities.to_csv(path + disease + '_top3_GDP_XGBoost.csv')